# Gerador de lat e log 

....

## importação dos dados

In [15]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from numpy import dtype

In [10]:
#importação de dados
df_original = pd.read_csv('../SISREG_Estudo_caso-AI_FALTAS_USUARIOS/dados/DADOS_SISREG.csv', 
    dtype={'CNES_EXECUTANTE':'str','IBGE_USUARIO':'str','CEP_USUARIO':'str', 'CEP_UNID_EXECUTANTE':'str'}) 
df_cep = pd.read_csv('../SISREG_Estudo_caso-AI_FALTAS_USUARIOS/dados/TABELA_CEP_BR.txt', dtype=str, sep="	", header=None)
df_cep.columns = ['CEP','UF','CIDADE','BAIRRO','LOGRADOURO']  #http://cep.la/baixar

## identificando CEP's unicos

### CEP's do USUARIO

In [3]:
#criando um DataFrame somente os CEP's dos usuarios.
df_ceps_usuario = df_original['CEP_USUARIO']
df_ceps_usuario = pd.DataFrame(df_ceps_usuario)
df_ceps_usuario.drop_duplicates(inplace=True)
df_ceps_usuario.dropna(inplace=True)
df_ceps_usuario.head(5)


,CEP_USUARIO
0,89180000
1,58410080
2,89687000
3,89896000
4,89745000


In [4]:
#quantidade de CEP's localizado na base 
print("Cep's unicos de usuarios: ",df_ceps_usuario.count())

Cep's unicos de usuarios:  CEP_USUARIO    173
dtype: int64


### CEP's da UNIDADE EXECUTANTE

In [5]:
#criando um DataFrame somente os CEP's das unidades.
df_ceps_cnes = df_original['CEP_UNID_EXECUTANTE']
df_ceps_cnes = pd.DataFrame(df_ceps_cnes)
df_ceps_cnes.drop_duplicates(inplace=True)
df_ceps_cnes.dropna(inplace=True)
df_ceps_cnes.head(5)

,CEP_UNID_EXECUTANTE
0,89180000
1,58428016
2,89687000
3,89896000
4,89745000


In [6]:
#quantidade de CEP's localizado na base e tratado
print("Cep's unicos de usuarios: ",df_ceps_cnes.count())

Cep's unicos de usuarios:  CEP_UNID_EXECUTANTE    40
dtype: int64


## TRATAMENTO COM OS CEP's DOS USUARIOS

In [11]:
#localizando endereço do usuario com a base do CEP's BR
df_ceps_usuario = pd.merge(df_ceps_usuario, df_cep, left_on=['CEP_USUARIO'],right_on=['CEP'] , how='left')

In [12]:
#criando um endereço completo para realizar pesquisa de lat e log
df_ceps_usuario["END_COMPLETO"] = df_ceps_usuario['LOGRADOURO']+ " - " + df_ceps_usuario['CIDADE']+ " - " + df_ceps_usuario['UF']
df_ceps_usuario.dropna(inplace=True)
df_ceps_usuario.head(5)

,CEP_USUARIO,CEP,UF,CIDADE,BAIRRO,LOGRADOURO,END_COMPLETO
1,58410080,58410080,PB,Campina Grande,Estação Velha,Rua João Caetano de Andrade,Rua João Caetano de Andrade - Campina Grande - PB
7,72855172,72855172,GO,Luziânia,Parque Industrial Mingone,Quadra Quadra 172,Quadra Quadra 172 - Luziânia - GO
12,72810520,72810520,GO,Luziânia,Setor Presidente Kennedy,Rua Paraná,Rua Paraná - Luziânia - GO
14,72800040,72800040,GO,Luziânia,Centro,Travessa Nirson Carneiro,Travessa Nirson Carneiro - Luziânia - GO
15,72815120,72815120,GO,Luziânia,Setor Mandu,Rua 31,Rua 31 - Luziânia - GO


In [16]:
#localizando lat e log (PROCESSO DEMORADO)
locator = Nominatim(user_agent='localizacao')
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df_ceps_usuario["GEOLOCALIZADO"] = df_ceps_usuario["END_COMPLETO"].apply(geocode)
df_ceps_usuario.head(10)

,CEP_USUARIO,CEP,UF,CIDADE,BAIRRO,LOGRADOURO,END_COMPLETO,GEOLOCALIZADO
1,58410080,58410080,PB,Campina Grande,Estação Velha,Rua João Caetano de Andrade,Rua João Caetano de Andrade - Campina Grande - PB,None
7,72855172,72855172,GO,Luziânia,Parque Industrial Mingone,Quadra Quadra 172,Quadra Quadra 172 - Luziânia - GO,"(Quadra 12, Valparaíso de Goiás, Região Geográ..."
12,72810520,72810520,GO,Luziânia,Setor Presidente Kennedy,Rua Paraná,Rua Paraná - Luziânia - GO,"(Rua Paraná, São Caetano, Luziânia, Região Geo..."
14,72800040,72800040,GO,Luziânia,Centro,Travessa Nirson Carneiro,Travessa Nirson Carneiro - Luziânia - GO,None
15,72815120,72815120,GO,Luziânia,Setor Mandu,Rua 31,Rua 31 - Luziânia - GO,"(Rua 31, Vila Guará, Luziânia, Região Geográfi..."
16,72803360,72803360,GO,Luziânia,Setor Leste,Rua Antônio Vieira Filho,Rua Antônio Vieira Filho - Luziânia - GO,None
17,78552349,78552349,MT,Sinop,Jardim das Oliveiras,Rua das Dombéias,Rua das Dombéias - Sinop - MT,None
18,06815640,06815640,SP,Embu das Artes,Conjunto Habitacional Embu N,Rua Ana Moysés,Rua Ana Moysés - Embu das Artes - SP,None
20,06843140,06843140,SP,Embu das Artes,Vila Marajoara,Rua Paiaguas,Rua Paiaguas - Embu das Artes - SP,"(Rua Paiaguás, Vila Santa Clara, Jardim Pinhei..."
22,58400090,58400090,PB,Campina Grande,Centro,Rua Venâncio Neiva,Rua Venâncio Neiva - Campina Grande - PB,"(Rua Venâncio Neiva, Centro, Campina Grande, R..."


In [17]:
#verificar LOCALIZAÇÃO lat e log não localizada (limpeza)
df_ceps_usuario.dropna(inplace=True)

In [18]:
#colocando a LATITUDE e LONGITUDE na tabela
df_ceps_usuario['LATITUDE_USUARIO'] = df_ceps_usuario['GEOLOCALIZADO'].apply(lambda loc: loc.latitude)
df_ceps_usuario['LONGITUDE_USUARIO'] = df_ceps_usuario['GEOLOCALIZADO'].apply(lambda loc: loc.longitude)
df_ceps_usuario.drop('END_COMPLETO', axis=1, inplace=True)
df_ceps_usuario.drop('GEOLOCALIZADO', axis=1, inplace=True)
df_ceps_usuario.head(10)

,CEP_USUARIO,CEP,UF,CIDADE,BAIRRO,LOGRADOURO,LATITUDE_USUARIO,LONGITUDE_USUARIO
7,72855172,72855172,GO,Luziânia,Parque Industrial Mingone,Quadra Quadra 172,-16.086994,-47.983412
12,72810520,72810520,GO,Luziânia,Setor Presidente Kennedy,Rua Paraná,-16.261027,-47.942692
15,72815120,72815120,GO,Luziânia,Setor Mandu,Rua 31,-16.228359,-47.911296
20,06843140,06843140,SP,Embu das Artes,Vila Marajoara,Rua Paiaguas,-23.680583,-46.865803
22,58400090,58400090,PB,Campina Grande,Centro,Rua Venâncio Neiva,-7.219468,-35.883776
23,72837090,72837090,GO,Luziânia,Parque Alvorada II,Rua 75,-16.156912,-47.954593
27,06816220,06816220,SP,Embu das Artes,Parque Luíza,Rua Campo Florido,-23.646898,-46.818152
31,72815610,72815610,GO,Luziânia,Parque JK,Rua 19,-16.279330,-47.859893
32,06803040,06803040,SP,Embu das Artes,Centro,Rua do Virgílio,-23.651426,-46.848613
33,58415270,58415270,PB,Campina Grande,Jardim Paulistano,Rua Doutor Edésio Silva,-7.249646,-35.891586


In [19]:
#quantidade de CEP's localizado na base e tratado
print("Cep's unicos de usuarios: ",df_ceps_usuario.count())

Cep's unicos de usuarios:  CEP_USUARIO          70
CEP                  70
UF                   70
CIDADE               70
BAIRRO               70
LOGRADOURO           70
LATITUDE_USUARIO     70
LONGITUDE_USUARIO    70
dtype: int64


## GERAÇÃO DE ARQUIVO CSV - CEP'S com latitude e longitude 

In [21]:
df_ceps_tratado = df_ceps_usuario.filter(items=['CEP','UF','CIDADE','BAIRRO','LOGRADOURO','LATITUDE_USUARIO','LONGITUDE_USUARIO'])
df_ceps_tratado

,CEP,UF,CIDADE,BAIRRO,LOGRADOURO,LATITUDE_USUARIO,LONGITUDE_USUARIO
7,72855172,GO,Luziânia,Parque Industrial Mingone,Quadra Quadra 172,-16.086994,-47.983412
12,72810520,GO,Luziânia,Setor Presidente Kennedy,Rua Paraná,-16.261027,-47.942692
15,72815120,GO,Luziânia,Setor Mandu,Rua 31,-16.228359,-47.911296
20,06843140,SP,Embu das Artes,Vila Marajoara,Rua Paiaguas,-23.680583,-46.865803
22,58400090,PB,Campina Grande,Centro,Rua Venâncio Neiva,-7.219468,-35.883776
...,...,...,...,...,...,...,...
165,72830080,GO,Luziânia,Parque Estrela Dalva VII,Rua 8,-16.179284,-47.931948
166,78557421,MT,Sinop,Residencial Lisboa,Rua Quatro,-11.818532,-55.478375
167,58408072,PB,Campina Grande,Vila Cabral,Rua São Luiz,-7.176637,-35.979719
171,06815000,SP,Embu das Artes,Parque Pirajussara,Avenida Aimará,-23.653574,-46.815139


In [22]:
df_ceps_tratado.to_csv('../SISREG_Estudo_caso-AI_FALTAS_USUARIOS/dados/TABELA_CEP_LAT_LON.csv')